# Processing

In [76]:
import pandas as pd
import plotly.express as px
import plotly.graph_objs as go


In [10]:
df = pd.read_csv("data_clean.csv")
df = df.drop("Unnamed: 0", axis=1)
df

,Squad,pts,G.,N.,P.,p.,c.,+/-,Championnat,ATT,...,Crs,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%
0,Milan,86,26,8,4,69,31,38,ITA,81.0,...,672,355,421,5,5,2,2167,521,517,50.2
1,Inter,84,25,9,4,84,32,52,ITA,82.0,...,809,332,332,7,4,1,2004,549,476,53.6
2,Napoli,79,24,7,7,74,31,43,ITA,82.0,...,553,296,327,9,1,1,1981,393,403,49.4
3,Juventus,70,20,10,8,57,37,20,ITA,81.0,...,550,332,316,5,7,1,2008,550,440,55.6
4,Lazio,64,18,10,10,77,58,19,ITA,79.0,...,574,315,339,6,6,1,1941,411,427,49.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,Mallorca,39,10,9,19,36,63,-27,ESP,75.0,...,597,344,332,6,8,4,2007,651,703,48.1
76,Cádiz,39,8,15,15,35,51,-16,ESP,77.0,...,865,413,380,5,13,0,2077,601,693,46.4
77,Granada,38,8,14,16,44,61,-17,ESP,77.0,...,691,326,343,2,9,2,1867,602,743,44.8
78,Levante,35,8,11,19,51,76,-25,ESP,78.0,...,624,377,305,6,6,0,2054,441,492,47.3


## New Order

In [6]:
df.sort_values("pts", ascending=False)

,Unnamed: 0,Squad,pts,G.,N.,P.,p.,c.,+/-,Championnat,...,Crs,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%
20,20,Manchester City,93,29,6,3,99,26,73,ANG,...,869,279,284,7,1,0,1967,482,397,54.8
21,21,Liverpool,92,28,8,2,94,26,68,ANG,...,897,340,320,5,0,1,2265,571,467,55.0
0,0,Milan,86,26,8,4,69,31,38,ITA,...,672,355,421,5,5,2,2167,521,517,50.2
60,60,Real Madrid,86,26,8,4,80,31,49,ESP,...,611,334,345,10,5,0,1941,397,319,55.4
40,40,Paris S-G,86,26,8,4,90,36,54,FRA,...,459,303,360,6,3,1,1825,262,221,54.2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17,17,Cagliari,30,6,12,20,34,68,-34,ITA,...,675,397,303,3,10,1,1995,696,742,48.4
18,18,Genoa,28,4,16,18,27,60,-33,ITA,...,661,449,416,4,5,2,2228,635,743,46.1
19,19,Venezia,27,6,9,23,34,69,-35,ITA,...,585,388,352,5,12,2,1942,564,583,49.2
38,38,Watford,23,6,5,27,34,77,-43,ANG,...,610,444,356,2,7,3,2154,711,749,48.7


In [55]:
def create_rank(col, df, asc):
    df = df.sort_values(col, ascending=asc)
    df = df.reset_index(drop=True)
    df.index = df.index.set_names(['rank_'+col])
    df = df.reset_index(drop=False)
    df["rank_"+col] = df["rank_"+col]+1
    return df

def ranking_per_country(country, coef):

    df_w = df[df["Championnat"]==country]
    df_w = create_rank("pts", df_w, False)
    df_w = create_rank("p.", df_w, False)
    df_w = create_rank("c.", df_w, True)
    df_w = create_rank("+/-", df_w, False)
    df_w = create_rank("Won%", df_w, False)
    df_w = create_rank("Fls", df_w, True)
    df_w = create_rank("CrdY", df_w, True)
    df_w = create_rank("AVG", df_w, False)

    df_w["rank_total"] = df_w.iloc[:, 0:8].sum(axis=1) 

    df_final = df_w.iloc[:, 0:9].copy()
    df_final["rank_avg"] = round(df_w["rank_total"]/8,1)
    df_final.sort_values("rank_avg", ascending=True)
    df_final["Championnat"] = country 

    df_final["rating_adj/20"] = round((20-df_final["rank_avg"])*coef,2)

    return df_final

In [61]:
df_FRA = ranking_per_country("FRA", 0.59497)
df_ITA = ranking_per_country("ITA", 0.71497)
df_ESP = ranking_per_country("ESP", 0.88855)
df_ANG = ranking_per_country("ANG", 1.03141)

result = pd.concat([df_ANG, df_ESP, df_FRA, df_ITA])
result = result.sort_values(by="rating_adj/20", ascending=False)
result = result[["Squad","rating_adj/20",	"rank_avg"	,"Championnat", 	"rank_AVG",	"rank_CrdY",	"rank_Fls",	"rank_Won%",	"rank_+/-",	"rank_c.",	"rank_p.",	"rank_pts"]]
result


,Squad,rating_adj/20,rank_avg,Championnat,rank_AVG,rank_CrdY,rank_Fls,rank_Won%,rank_+/-,rank_c.,rank_p.,rank_pts
0,Manchester City,19.39,1.2,ANG,1,1,1,3,1,1,1,1
1,Liverpool,18.15,2.4,ANG,2,2,5,2,2,2,2,2
0,Real Madrid,16.70,1.2,ESP,1,1,1,2,1,2,1,1
2,Chelsea,14.54,5.9,ANG,3,8,17,7,3,3,3,3
6,West Ham,14.23,6.2,ANG,7,3,2,9,6,9,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...
17,Mallorca,2.31,17.4,ESP,18,18,18,15,19,18,17,16
17,Genoa,2.29,16.8,ITA,18,11,19,17,17,13,20,19
12,Leeds United,2.17,17.9,ANG,13,20,20,20,18,19,16,17
12,Saint-Étienne,1.84,16.9,FRA,13,16,14,20,19,19,16,18


In [62]:
result.to_csv("data_ranked.csv")

In [82]:
df_agg = df.groupby("Championnat").mean()
df_agg.reset_index(inplace=True)
df_agg

,Championnat,pts,G.,N.,P.,p.,c.,+/-,ATT,MIL,...,Crs,Int,TklW,PKwon,PKcon,OG,Recov,Won,Lost,Won%
0,ANG,52.60,14.60,8.8,14.60,53.55,53.55,0.0,78.65,78.60,...,679.50,365.60,357.00,3.85,5.15,1.70,2040.40,585.9,585.9,50.075
1,ESP,51.45,13.45,11.1,13.45,47.55,47.55,0.0,78.85,77.90,...,710.90,349.35,341.45,4.55,6.65,1.20,2002.25,587.5,587.5,50.050
2,FRA,51.80,13.90,10.2,13.90,53.35,53.35,0.0,75.80,74.75,...,657.40,413.30,376.90,4.55,6.10,1.80,2056.00,480.0,480.1,50.085
3,ITA,52.10,14.10,9.8,14.10,54.45,54.45,0.0,76.80,75.75,...,637.45,365.20,333.45,5.00,7.10,1.85,2037.95,548.9,548.9,50.065


### Niveau Théorique

In [92]:
px.scatter(df, x="ATT", y="DEF", color="Championnat", hover_name="Squad")


In [91]:
px.bar(df_agg, x="Championnat", y=["DEF", "ATT"])


### Fair-Play

In [93]:
px.scatter(df, x="Fls", y="CrdY", color="Championnat", hover_name="Squad")

In [94]:
px.bar(df_agg, x="Championnat", y=["CrdY", "Fls"])


### Comparaison de notre note et du classement réel

In [98]:
fig = px.scatter(result, x="rank_avg", y="rank_pts", color="Championnat", hover_name="Squad")
fig.update_layout(
    shapes=[
        {
            'type': 'line',
            'x0': 1,
            'y0': 1,
            'x1': 20,
            'y1': 20,
            'line': {
                'color': 'black',
                'width': 1.5
            }
        }
    ])
fig.show()




In [100]:
px.scatter(result, x="rating_adj/20", y="rank_pts", color="Championnat", hover_name="Squad")

In [95]:
result

,Squad,rating_adj/20,rank_avg,Championnat,rank_AVG,rank_CrdY,rank_Fls,rank_Won%,rank_+/-,rank_c.,rank_p.,rank_pts
0,Manchester City,19.39,1.2,ANG,1,1,1,3,1,1,1,1
1,Liverpool,18.15,2.4,ANG,2,2,5,2,2,2,2,2
0,Real Madrid,16.70,1.2,ESP,1,1,1,2,1,2,1,1
2,Chelsea,14.54,5.9,ANG,3,8,17,7,3,3,3,3
6,West Ham,14.23,6.2,ANG,7,3,2,9,6,9,7,7
...,...,...,...,...,...,...,...,...,...,...,...,...
17,Mallorca,2.31,17.4,ESP,18,18,18,15,19,18,17,16
17,Genoa,2.29,16.8,ITA,18,11,19,17,17,13,20,19
12,Leeds United,2.17,17.9,ANG,13,20,20,20,18,19,16,17
12,Saint-Étienne,1.84,16.9,FRA,13,16,14,20,19,19,16,18
